In [1]:
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
folder = "env"
sourcedir = currentdir.split(folder)[0]
sys.path.insert(0, sourcedir)
# from env.equation.equation import Equation
print(sourcedir)

/home/valdecar/Documents/projects/projectsNew/lab/hybriddomain/spaces/math_space/common/


### Добавление нового терма
Вызов метода (свойства) объекта: `a.t, a.t(), a.t(f), a.t().t()` 

Чтобы добавить новый терм в Equation нужно:</br>
> 1) Добавить input pattern в data/terms/input/wolfram/patterns

> 2) Добавить terms_for_args в data/terms/args

> 3) Добавить slambda pattern в data/terms/slambda/sympy/patterns

Второй шаг необходим, поскольку терм a.t содержит аргумент (a) и не является скобочным (извлечение аргументов из скобочных термов производиться автоматически при преобразовании дерева разбора в дерево операций). Можно было бы сделать терм a.t тоже скобочным, но тогда он должен бы иметь вид t(a). 
<br>
Отделение аргумента необходимо т.к. мы хотим получить возможность подстановки типа<br>
> `Equation("a.t() = a").subs(a=sympy.Matrix([[0,1],[-1, 0]]))` 

Иначе при подстановке заменяться будет все выражение a.t()

##### 1) Добавление input pattern

Для этого создается класс в data/terms/input/wolfram/patterns в методе `__call__` которого должен возвращаться re pattern:<br>
```

    def __init__(self, net):
        self.net = net
        self.id = 'dot'
        self.init_pattern()

    def init_pattern(self):
        
        self.obj = "(?P<obj>\w+)"
        self.arg = "(?P<arg>\w+)?"

        # find ()
        self.call = "(?P<call>\(%s\))?" % (self.arg)
    
        # find a.t, a.t(), a.t(), a.t().c()
        # but not a.1(), a.1:
        self.main = r"%s(\.([a-zA-Z])+%s)+" % (self.obj, self.call)
        
        self.gen = lambda: self.main

    def __call__(self):
        return(self.gen())
```
При задании re patterna важно, чтобы он не находил другие похожие термы. В данном случае если вместо [a-zA-Z] в `self.main` подставить \w то tokenizer будет также распозновать терм float (1.1). <br>
Важно также задать идентификатор терма (`self.id='dot'`)<br><br>

После создания patterna необходимо добавить его в три списка файла `data/terms/input/wolfram/lex_net_wolfram.py`: 
> terms_gens - для tokenizera:<br>
>>        terms_gens = [Base, ArgInt, ArgFloat, ArgDelay, ArgTime,
                      Var, VarBdp, Coeffs, Bdp, Diff, Pow, Func,
                      FreeVar, Time, DiffTimeVar, Dot]


> patterns_order - для приоритета терма с другими   
>>        patterns_order = ['diff',
                          'bdp',
                          'dot',
                          'func',
                          'diff_time',
                          'var',
                          'free_var',
                          'time',
                          'coeffs',
                          'pow',
                          'float']

> map_ptg - отображения lex терма в терм грамматики
>>         map_ptg = dict([('diff', 'a'),
                        ('bdp', 'a'),
                        ('diff_time', 'a'),
                        ('var', 'a'),
                        ('free_var', 'a'),
                        ('time', 'a'),
                        ('coeffs', 'a'),
                        ('pow', 'w'),
                        ('func', 'f'),
                        ('float', 'a'),
                        ('dot', 'a')])

После этого уже можно построить дерево операций:

In [3]:
from env.equation.equation import Equation
eq = Equation("a.t() = a")
eq.parser.parse()
print(eq.eq_tree)


=
child 0: br
   child 0: (
   child 1: args
      child 0: a
   child 2: )
child 1: br
   child 0: (
   child 1: args
      child 0: a
   child 2: )


Однако терм dot здесь представлен его значением в грамматике (map_ptg образ):

In [4]:
print(eq.eq_tree[1][1][0].name.lex)

['a.t()', <_sre.SRE_Match object; span=(1, 6), match='a.t()'>, 'dot']


##### 2) Отделение терма от его аргумента:

Чтобы выделить аргумент из вершины в отдельную вершину графа нужно добавить терм в
 data/terms/args/terms_for_args:<br>
> ```terms_for_args = {'dot':
                       {'child_name': lambda node: node.name.lex[1].group('obj'),
                        'child_term_id': 'free_var',
                        'editor': dot_editor}}
  ```

Здесь child_name child_term_id это значение и идентификатор будующей вершины в дереве, dot_editor используется для замены значения исходного терма с a.t() на .t()
```
def dot_editor(self, node):

    '''remove arg from node (a.t -> .t)
    for output generator'''
    
    val = node.name.lex[0]
    obj = node.name.lex[1].group('obj')
    node.name.lex[0] = val.replace(obj, "")
```

После этого разделение происходит в `get_args`  с помощью `ArgGen` файле  `data/terms/args/extractor`:

In [5]:
eq.args_editor.get_args()
print("tree:")
print(eq.eq_tree[1])
print("\nnodes:")
print(eq.eq_tree[1].name.lex)
print(eq.eq_tree[1][0].name.lex)

tree:
a
child 0: a

nodes:
['.t()', <_sre.SRE_Match object; span=(1, 6), match='a.t()'>, 'dot']
['a', None, 'free_var']


##### 3) Добавление sympy представления:

Для этого нужно добавить данные терма в `terms_gens_id` в файл `data/terms/slambda/sympy/patterns/ids/_list_ids.py`. Здесь в основном все сводиться к выражению:
> out = lambda X: X.transpose()

в классе `DotTranspose` в файле `data/terms/slambda/sympy/patterns/ids/dot.py`<br><br>
Теперь можно выполнить выражение:

In [6]:
import sympy

eq.args_editor.get_vars()

a = sympy.Matrix([[0, -1], [1, 0]])
eq.args_editor.subs(a=a)

eq.slambda.sympy.lambdify_sem()
out_a = eq.slambda.sympy.lambdify()
print("slambda:")
print(out_a)

b = sympy.Matrix([[0, 1], [1, 0]])
eq.args_editor.subs(a=b)

eq.slambda.sympy.lambdify_sem()
out_b = eq.slambda.sympy.lambdify()

print("\nresult for:")
print(a)
print("is:")
print(out_a())

print("\nresult for:")
print(b)
print("is:")
print(out_b())

eq.replacer.sympy.make_sympy()

print("\nsympy")
print(eq.tree.flatten('sympy'))


slambda:
<function EqSlambda.lambdify.<locals>.<lambda> at 0x7f0d0ebd57b8>

result for:
Matrix([[0, -1], [1, 0]])
is:
False

result for:
Matrix([[0, 1], [1, 0]])
is:
True

sympy
a.t()=(a)


При генерации последнего выражения использовалось оригинальное значение (если нет терма в `data/terms/output/sympy/patterns`)